In [ ]:
# default_exp glp.data

%reload_ext autoreload
%autoreload 2

In [2]:
from nbdev import *

# glp.data

A collection of classes, utility functions, etc. to handle the obnoxiousness of biological data.

In [3]:
import sys
sys.path.append('../')

In [4]:
#export

from fastai.text import *

import pandas as pd
import numpy as np


## Fasta Files

The most common dataformat for sequence info is a Fasta file.

```
>seq1
MRATCRA
>seq2
MRATTRA
```

This often needs to get paired with phenotype information held in the fasta file itself (in the header) or using the header as a key to match information in the csv file. This pipeline is a modular tool for performing some of these common functions and creating easy to use files for downstream processing.

This is structured in a modular way for easy reusability.

So an `AbstractRecordExtractor` should:
 - defines a `__call__` method that takes a seq_record and can recieve **kwargs from previous transforms.


In [5]:
#export

class AbstractRecordExtractor(object):
    
    
    def __call__(self, seqR, **kwargs):
        raise NotImplementedError



The simplest of which is likely the `SeqExtractor` which retreives the sequence from the record and optionally truncates it.

In [6]:
# export
class SeqExtractor(AbstractRecordExtractor):
    
    def __init__(self, field = 'sequence', truncate = None, ungap = False):
        
        self.field = field
        self.truncate = truncate
        self.ungap = ungap
        
    def __call__(self, seqR, **kwargs):
        
        seq = str(seqR.seq)
        if self.ungap:
            seq = seq.replace('-', '')
            
        if self.truncate is not None:
            seq = seq[:self.truncate]
            
        return {self.field: seq}

In [7]:
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq

seq_trial = SeqRecord(Seq('ACGTACGT'), id = 'SeqID')

seq_ext = SeqExtractor()
features = seq_ext(seq_trial)
assert 'sequence' in features
assert features['sequence'] == 'ACGTACGT'


seq_ext = SeqExtractor(truncate=4)
features = seq_ext(seq_trial)
assert 'sequence' in features
assert features['sequence'] == 'ACGT'



And something to read and process the id.

In [8]:
#export

class IDExtractor(AbstractRecordExtractor):
    
    def __init__(self, field = 'id', keyfunc = None, returns_dict = False):
        
        self.field = field
        self.returns_dict = returns_dict
        self.keyfunc = keyfunc if keyfunc else lambda x: x
        
    def __call__(self, seqR, **kwargs):
        
        _id = seqR.id
        res = self.keyfunc(_id)
        if self.returns_dict:
            return res
        else:
            return {self.field: res}
        
        

In [9]:
seq_trial = SeqRecord(Seq('ACGTACGT'), id = 'SeqID')

seq_ext = IDExtractor()
features = seq_ext(seq_trial)
assert 'id' in features
assert features['id'] == 'SeqID'


seq_ext = IDExtractor(keyfunc = lambda _id: _id.lower())
features = seq_ext(seq_trial)
assert 'id' in features
assert features['id'] == 'seqid'

def split_func(_id):
    return {'first': _id[:3], 'last': _id[-2:]}

seq_ext = IDExtractor(keyfunc = split_func, returns_dict=True)
features = seq_ext(seq_trial)
assert ('first' in features) and ('last' in features)
assert (features['first'] == 'Seq') and (features['last'] == 'ID')



These two processes cover >90% of use cases.
Let's see how to combine these into a `FastaPipeline` to pre-process fasta files into easier to use csv files.

In [10]:
#export

from itertools import chain


class FastaPipeline(object):
    
    def __init__(self, extractors):
        
        self.extractors = extractors
        
    
    def process_seqrecord(self, seqR):
        
        info = {}
        for ext in self.extractors:
            info.update(ext(seqR, **info))
        
        return info
    
    
    def process_seq_stream(self, stream):
        
        seq_data = []
        for seqR in stream:
            seq_data.append(self.process_seqrecord(seqR))
        
        seq_df = pd.DataFrame(seq_data)
        
        return seq_df
    
    def fasta2df(self, path = None, stream = None, 
                 feature_data = None, merge_keys = None,
                 grouper = None):
        
        if stream is None:
            if type(path) == str:
                stream = SeqIO.parse(open(path), 'fasta')
            else:
                stream = chain.from_iterable(SeqIO.parse(open(p), 'fasta') for p in path)
            
        seq_df = self.process_seq_stream(stream)
        
        if feature_data is not None:
            assert merge_keys is not None, 'If feature_data is provided merge_keys must be provided'
            feature_on, seq_on = merge_keys
            res = pd.merge(feature_data, seq_df,
                           left_on = feature_on, right_on = seq_on)
        else:
            res = seq_df
            
        if grouper is not None:
            res = res.groupby(grouper, as_index = False).first()
        
        return res


In [11]:
seq_stream = [SeqRecord(Seq('ACGTACGT'), id = 'SeqID1'),
              SeqRecord(Seq('TGCATGCA'), id = 'SeqID2')]

df = pd.DataFrame([{'idl': 'seqid1', 'feature': 1},
                   {'idl': 'seqid2', 'feature': 2},
                   ])

pipeline = FastaPipeline([SeqExtractor(), 
                          IDExtractor(keyfunc=lambda _id: _id.lower())])

res = pipeline.fasta2df(stream = seq_stream, 
                        feature_data=df,
                        merge_keys = ('idl', 'id'))
pd.testing.assert_series_equal(res['feature'], pd.Series([1, 2]), check_names=False)
pd.testing.assert_series_equal(res['sequence'], pd.Series(['ACGTACGT', 'TGCATGCA']), check_names=False)
res

,idl,feature,sequence,id
0,seqid1,1,ACGTACGT,seqid1
1,seqid2,2,TGCATGCA,seqid2
